1) Next Permutation
Description
You have an array (or string) representing a permutation of its elements. You want to modify it in-place to produce the next lexicographically greater permutation. If no such permutation exists (i.e. it’s the highest lexicographic order), then you transform it into the lowest (sorted ascending) permutation.

Example:

Input: [1,2,3] → Output: [1,3,2]
Input: [3,2,1] → Output: [1,2,3] (wrap around)
Solution Approach
Find rightmost ascending pair: Traverse from the end to find the first index i such that arr[i] < arr[i+1]. If none found, reverse entire array.
Swap: From the end, find the first index j s.t. arr[j] > arr[i]. Swap arr[i] and arr[j].
Reverse the subarray from i+1 to the end.
Time Complexity: O(n)

Test Cases
[1,2,3] → [1,3,2]
[3,2,1] → [1,2,3]
[1,1,5] → [1,5,1]
[1,3,2] → [2,1,3]

In [1]:
def next_permutation(arr):
    n = len(arr)
    # 1. Find pivot
    pivot = -1
    for i in range(n-2, -1, -1):
        if arr[i] < arr[i+1]:
            pivot = i
            break
    
    if pivot == -1:
        # no pivot => highest permutation => reverse
        arr.reverse()
        return
    
    # 2. Find successor from right
    for j in range(n-1, pivot, -1):
        if arr[j] > arr[pivot]:
            arr[pivot], arr[j] = arr[j], arr[pivot]
            break
    
    # 3. Reverse suffix
    arr[pivot+1:] = reversed(arr[pivot+1:])


if __name__ == "__main__":
    test_cases = [
        [1,2,3],
        [3,2,1],
        [1,1,5],
        [1,3,2]
    ]
    for t in test_cases:
        arr = t[:]
        next_permutation(arr)
        print(t, "=>", arr)


[1, 2, 3] => [1, 3, 2]
[3, 2, 1] => [1, 2, 3]
[1, 1, 5] => [1, 5, 1]
[1, 3, 2] => [2, 1, 3]


In [ ]:
#C
#include <stdio.h>

void reverse(int* arr, int start, int end) {
    while(start < end) {
        int temp = arr[start];
        arr[start] = arr[end];
        arr[end] = temp;
        start++;
        end--;
    }
}

void nextPermutation(int* arr, int n) {
    // 1. Find pivot
    int pivot = -1;
    for(int i=n-2; i>=0; i--){
        if(arr[i] < arr[i+1]){
            pivot = i;
            break;
        }
    }
    if(pivot == -1){
        // reverse entire array
        reverse(arr, 0, n-1);
        return;
    }
    // 2. Find successor
    for(int j=n-1; j>pivot; j--){
        if(arr[j] > arr[pivot]){
            int temp = arr[j];
            arr[j] = arr[pivot];
            arr[pivot] = temp;
            break;
        }
    }
    // 3. Reverse suffix
    reverse(arr, pivot+1, n-1);
}

int main(){
    int test1[] = {1,2,3};  int n1=3;
    nextPermutation(test1,n1); // => [1,3,2]
    for(int i=0;i<n1;i++) printf("%d ", test1[i]);
    printf("\n");
    
    int test2[] = {3,2,1};  int n2=3;
    nextPermutation(test2,n2); // => [1,2,3]
    for(int i=0;i<n2;i++) printf("%d ", test2[i]);
    printf("\n");
    
    return 0;
}

2) K-th Permutation
Description
Given n (1 ≤ n ≤ 9) and a number k, find the k-th permutation of the numbers [1,2,...,n] in lexicographic order. If k is out of range, handle accordingly (some variants return an empty string or an error).

Example:

n=3, k=3 => permutations in sorted order:
1 2 3
1 3 2
2 1 3 <-- the 3rd
2 3 1
3 1 2
3 2 1
Solution Approach
We can use factorial number system logic:
Precompute factorials up to n.
Keep a list of digits [1..n].
For each position i from 0 to n-1:
compute index = k // factorial[n-1 - i], then pick that index from the remaining digits.
remove that digit from the list. update k %= factorial[n-1 - i].
Time Complexity: O(n^2) since removing from a list can be O(n).

Test Cases
n=3, k=1 => 1 2 3
n=3, k=3 => 2 1 3
n=4, k=9 => permutations of [1,2,3,4], the 9th is 2 3 1 4
n=3, k=7 => out of range if n=3 => max permutations=6, so handle gracefully.

In [2]:
def kth_permutation(n, k):
    import math
    nums = list(range(1, n+1))
    factorial = [1]*(n+1)
    for i in range(1,n+1):
        factorial[i] = factorial[i-1]*i

    if k > factorial[n]:
        return ""  # or some error handling

    k -= 1  # zero-based index
    result = []
    for i in range(n,0,-1):
        f = factorial[i-1]
        index = k // f
        result.append(str(nums[index]))
        nums.pop(index)
        k %= f
    return " ".join(result)

if __name__ == "__main__":
    print(kth_permutation(3,1))  # "1 2 3"
    print(kth_permutation(3,3))  # "2 1 3"
    print(kth_permutation(4,9))  # "2 3 1 4"
    print(kth_permutation(3,7))  # out of range => ""

1 2 3
2 1 3
2 3 1 4



In [ ]:
#C
#include <stdio.h>
#include <stdlib.h>

char* kthPermutation(int n, int k) {
    // factorial
    int* fact = (int*)malloc((n+1)*sizeof(int));
    fact[0] = 1;
    for(int i=1; i<=n; i++){
        fact[i] = fact[i-1]*i;
    }
    if(k>fact[n]) {
        free(fact);
        // return empty string
        char* empty = (char*)malloc(1);
        empty[0]='\0';
        return empty;
    }
    // build list [1..n]
    int* nums = (int*)malloc(n*sizeof(int));
    for(int i=0;i<n;i++){
        nums[i] = i+1;
    }
    k--; // 0-based
    // we'll store result in buffer
    char* result = (char*)malloc(n*5+1); // enough for "1 2 3 4 ..." + null
    result[0] = '\0';
    int used = 0; // length of result string

    for(int i=n;i>=1;i--){
        int f = fact[i-1];
        int index = k / f;
        // pick nums[index]
        used += sprintf(result+used, "%d", nums[index]);
        if(i>1) {
            used += sprintf(result+used, " ");
        }
        // remove nums[index]
        for(int j=index; j<(i-1); j++){
            nums[j] = nums[j+1];
        }
        k %= f;
    }
    result[used] = '\0';
    free(nums);
    free(fact);
    return result;
}

int main(){
    char* r = kthPermutation(3,3); // "2 1 3"
    printf("%s\n", r);
    free(r);
    r = kthPermutation(4,9); // "2 3 1 4"
    printf("%s\n", r);
    free(r);
    r = kthPermutation(3,7); // out of range => ""
    printf("'%s'\n", r);
    free(r);
    return 0;
}

3) Permutations of a String/Array with Duplicates
Description
Given a collection of elements (string or array) that may contain duplicates, generate all unique permutations. For instance, for [1,1,2], the unique permutations are:

[1,1,2]
[1,2,1]
[2,1,1]
Solution Approach
Standard backtracking to generate permutations, but:
Sort the array first.
When generating permutations, skip duplicates: if arr[i] == arr[i-1] and i-1 was not used, skip i.
Time Complexity: O(n! * n)

Test Cases
[1,2,3] => 6 permutations
[1,1,2] => 3 permutations (above)
[2,2,2] => 1 permutation

In [3]:
def permute_unique(nums):
    nums.sort()
    results = []
    used = [False]*len(nums)

    def backtrack(path):
        if len(path) == len(nums):
            results.append(path[:])
            return
        for i in range(len(nums)):
            if used[i]:
                continue
            # skip duplicates
            if i>0 and nums[i] == nums[i-1] and not used[i-1]:
                continue
            used[i] = True
            path.append(nums[i])
            backtrack(path)
            path.pop()
            used[i] = False

    backtrack([])
    return results

if __name__ == "__main__":
    print(permute_unique([1,1,2]))  # [[1,1,2],[1,2,1],[2,1,1]]
    print(permute_unique([2,2,2]))  # [[2,2,2]]


[[1, 1, 2], [1, 2, 1], [2, 1, 1]]
[[2, 2, 2]]


In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <stdbool.h>

int cmp(const void* a, const void* b){ return (*(int*)a - *(int*)b); }

bool used[100];

void backtrack(int* nums, int n, int* path, int depth, bool* used, int** results, int* returnSize, int** colSizes) {
    if(depth == n){
        results[*returnSize] = (int*)malloc(n*sizeof(int));
        for(int i=0;i<n;i++){
            results[*returnSize][i] = path[i];
        }
        colSizes[*returnSize][0] = n;
        (*returnSize)++;
        return;
    }
    for(int i=0;i<n;i++){
        if(used[i]) continue;
        // skip duplicates
        if(i>0 && nums[i]==nums[i-1] && !used[i-1]) continue;
        used[i] = true;
        path[depth] = nums[i];
        backtrack(nums,n,path,depth+1,used,results,returnSize,colSizes);
        used[i] = false;
    }
}

int main(){
    int arr[] = {1,1,2};
    int n=3;
    qsort(arr,n,sizeof(int),cmp);

    // max permutations = n! => for safety let's allocate
    int** results = (int**)malloc(6*sizeof(int*));
    int** colSizes = (int**)malloc(6*sizeof(int*));
    for(int i=0;i<6;i++){
        colSizes[i] = (int*)malloc(sizeof(int));
    }
    int returnSize=0;
    int path[100];

    backtrack(arr,n,path,0,used,results,&returnSize,colSizes);
    for(int i=0;i<returnSize;i++){
        for(int j=0;j<colSizes[i][0]; j++){
            printf("%d ", results[i][j]);
        }
        printf("\n");
    }
    return 0;
}


4) Permutation Index (Rank of a Permutation)
Description
Given a unique permutation P of [1..n], find its 1-based rank among all permutations in sorted order. For example, [1,2,3] has rank 1, [1,3,2] has rank 2, [2,1,3] has rank 3, etc.

Solution Approach
For each position i in P, count how many elements to the right are smaller (for ascending rank).
Weighted by factorial. If at index i the count is c, then contribution to rank is c * factorial[n-1-i]. Summation plus 1 for 1-based.
Test Cases
[1,2,3] => rank=1
[1,3,2] => rank=2
[3,1,2] => rank=5
[2,1,3] => rank=3


In [4]:
def permutation_index(perm):
    n = len(perm)
    # Precompute factorial
    fact = [1]*(n+1)
    for i in range(1,n+1):
        fact[i] = fact[i-1]*i

    rank = 1
    for i in range(n):
        smaller = 0
        for j in range(i+1,n):
            if perm[j] < perm[i]:
                smaller += 1
        rank += smaller * fact[n-1-i]
    return rank

if __name__ == "__main__":
    print(permutation_index([1,2,3]))  # 1
    print(permutation_index([1,3,2]))  # 2
    print(permutation_index([3,1,2]))  # 5
    print(permutation_index([2,1,3]))  # 3


1
2
5
3


In [ ]:
#include <stdio.h>

long long factorial[21];

long long permutationIndex(int* perm, int n){
    factorial[0]=1;
    for(int i=1;i<=n;i++){
        factorial[i] = factorial[i-1]*i;
    }
    long long rank=1;
    for(int i=0; i<n; i++){
        int smaller=0;
        for(int j=i+1;j<n;j++){
            if(perm[j]<perm[i]) smaller++;
        }
        rank += smaller*factorial[n-1-i];
    }
    return rank;
}

int main(){
    int p1[] = {1,2,3};
    printf("%lld\n", permutationIndex(p1,3)); // 1

    int p2[] = {3,1,2};
    printf("%lld\n", permutationIndex(p2,3)); // 5

    return 0;
}


5) Permutation in String (LeetCode 567)
Description
Given two strings s1 and s2, check if any permutation of s1 is a substring of s2. Return true if such a substring exists, otherwise false.

Example:

s1 = "ab", s2 = "eidbaooo" => true because "ba" is a substring of s2.
Solution Approach
Use a sliding window of length len(s1) over s2 and track the character counts vs. s1’s counts. If at any point the counts match, return true.

Test Cases
s1="ab", s2="eidbaooo" => true ("ba")
s1="ab", s2="eidboaoo" => false
s1="adc", s2="dcda" => true (substring "cda" is a permutation "adc")

In [5]:
def check_inclusion(s1, s2):
    from collections import Counter
    n1, n2 = len(s1), len(s2)
    if n1>n2: return False
    
    count1 = Counter(s1)
    window = Counter(s2[:n1])
    if window == count1:
        return True
    
    for i in range(n1, n2):
        window[s2[i]] += 1
        left_char = s2[i-n1]
        window[left_char] -= 1
        if window[left_char] == 0:
            del window[left_char]
        if window == count1:
            return True
    return False

if __name__ == "__main__":
    print(check_inclusion("ab","eidbaooo"))  # true
    print(check_inclusion("ab","eidboaoo"))  # false


True
False


In [ ]:
#include <stdio.h>
#include <stdbool.h>
#include <string.h>

bool checkInclusion(const char* s1, const char* s2){
    int n1=strlen(s1), n2=strlen(s2);
    if(n1>n2) return false;
    int count1[26]={0}, window[26]={0};
    for(int i=0;i<n1;i++){
        count1[s1[i]-'a']++;
        window[s2[i]-'a']++;
    }
    // compare function
    auto compare = [&](int* a, int* b){
        for(int k=0;k<26;k++){
            if(a[k]!=b[k]) return false;
        }
        return true;
    };
    if(compare(count1, window)) return true;
    
    for(int i=n1; i<n2;i++){
        window[s2[i]-'a']++;
        window[s2[i-n1]-'a']--;
        if(compare(count1,window)) return true;
    }
    return false;
}

// We'll do a small lambda emulation:
bool compareArrays(int* a, int* b){
    for(int i=0;i<26;i++){
        if(a[i]!=b[i]) return false;
    }
    return true;
}

bool checkInclusionC(const char* s1, const char* s2){
    int n1=strlen(s1),n2=strlen(s2);
    if(n1>n2) return false;
    int count1[26]={0}, window[26]={0};
    for(int i=0;i<n1;i++){
        count1[s1[i]-'a']++;
        window[s2[i]-'a']++;
    }
    if(compareArrays(count1,window)) return true;
    for(int i=n1;i<n2;i++){
        window[s2[i]-'a']++;
        window[s2[i-n1]-'a']--;
        if(compareArrays(count1,window)) return true;
    }
    return false;
}

int main(){
    printf("%d\n", checkInclusionC("ab","eidbaooo")); // 1 (true)
    printf("%d\n", checkInclusionC("ab","eidboaoo")); // 0 (false)
    return 0;
}


6) Longest Substring Without Repeating Characters
Description
Given a string s, find the length of the longest substring in which all characters are distinct.

Example:

s = "abcabcbb" => longest substring w/o repeats is "abc", length=3.
s = "pwwkew" => "wke" length=3.
Solution Approach
Use a sliding window + hash/array of freq or last indices:

Expand end, track the last occurrences.
If a character is repeated, move start accordingly.
Keep track of max window size.
Test Cases
"abcabcbb" => 3
"bbbbb" => 1
"pwwkew" => 3
"" => 0

In [6]:
def length_of_longest_substring(s):
    last_index = {}
    start = 0
    max_len = 0
    for i, ch in enumerate(s):
        if ch in last_index and last_index[ch] >= start:
            start = last_index[ch] + 1
        last_index[ch] = i
        max_len = max(max_len, i - start + 1)
    return max_len

if __name__ == "__main__":
    print(length_of_longest_substring("abcabcbb")) # 3
    print(length_of_longest_substring("bbbbb"))    # 1
    print(length_of_longest_substring("pwwkew"))   # 3


3
1
3


In [ ]:
#include <stdio.h>
#include <string.h>

int lengthOfLongestSubstring(const char* s){
    int last[256];
    for(int i=0;i<256;i++) last[i] = -1;
    int start=0, maxLen=0;
    for(int i=0; s[i]; i++){
        unsigned char c = s[i];
        if(last[c]>=start){
            start = last[c]+1;
        }
        last[c] = i;
        int length = i - start + 1;
        if(length>maxLen) maxLen=length;
    }
    return maxLen;
}

int main(){
    printf("%d\n", lengthOfLongestSubstring("abcabcbb")); // 3
    printf("%d\n", lengthOfLongestSubstring("bbbbb"));    // 1
    printf("%d\n", lengthOfLongestSubstring("pwwkew"));   // 3
    return 0;
}

7) Minimum Window Substring
Description
Given two strings s and t, find the minimum window in s which contains all the characters of t. If no such window exists, return "".

Example:

s = "ADOBECODEBANC", t = "ABC" => The minimum window containing A,B,C is "BANC" (length=4).
Solution Approach
Sliding window + character counts:

Count freq of each character in t.
Expand end pointer, track how many characters matched.
When matched all needed chars, try to shrink window from start.
Keep track of min-length window.
Time Complexity: O(|s| + |t|)

Test Cases
s="ADOBECODEBANC", t="ABC" => "BANC"
s="a", t="a" => "a"
s="a", t="b" => ""

In [7]:
from collections import Counter

def min_window(s, t):
    if not s or not t:
        return ""
    countT = Counter(t)
    required = len(countT)
    have = 0
    window_count = {}
    res = (float("inf"), 0, 0)
    left = 0

    for right, char in enumerate(s):
        window_count[char] = window_count.get(char, 0) + 1
        if char in countT and window_count[char] == countT[char]:
            have += 1

        while have == required:
            # update result
            if (right - left + 1) < res[0]:
                res = (right-left+1, left, right)
            # shrink from left
            left_char = s[left]
            window_count[left_char] -= 1
            if left_char in countT and window_count[left_char] < countT[left_char]:
                have -= 1
            left += 1

    if res[0] == float("inf"):
        return ""
    return s[res[1]:res[2]+1]

if __name__ == "__main__":
    print(min_window("ADOBECODEBANC","ABC"))  # "BANC"
    print(min_window("a","a"))               # "a"
    print(min_window("a","b"))               # ""


BANC
a



In [ ]:
#include <stdio.h>
#include <string.h>
#include <limits.h>

char* minWindow(const char* s, const char* t){
    int ls=strlen(s), lt=strlen(t);
    if(!ls || !lt) {
        char* empty=(char*)malloc(1);
        empty[0]='\0';
        return empty;
    }
    int freqT[256]={0}, window[256]={0};
    for(int i=0;i<lt;i++){
        freqT[(unsigned char)t[i]]++;
    }
    int required=0;
    for(int i=0;i<256;i++){
        if(freqT[i]>0) required++;
    }
    int formed=0; 
    int left=0, right=0;
    int minLen=INT_MAX, startIndex=0;
    
    while(right<ls){
        unsigned char c=(unsigned char)s[right];
        window[c]++;
        if(freqT[c]>0 && window[c]==freqT[c]){
            formed++;
        }
        while(left<=right && formed==required){
            // update minLen
            int length = right-left+1;
            if(length<minLen){
                minLen=length;
                startIndex=left;
            }
            // pop from left
            unsigned char lc=(unsigned char)s[left];
            window[lc]--;
            if(freqT[lc]>0 && window[lc]<freqT[lc]){
                formed--;
            }
            left++;
        }
        right++;
    }
    if(minLen==INT_MAX){
        char* empty=(char*)malloc(1);
        empty[0]='\0';
        return empty;
    }
    // build result
    char* result=(char*)malloc(minLen+1);
    strncpy(result, s+startIndex, minLen);
    result[minLen]='\0';
    return result;
}

int main(){
    char* r=minWindow("ADOBECODEBANC","ABC"); // "BANC"
    printf("%s\n", r);
    free(r);

    r=minWindow("a","a"); // "a"
    printf("%s\n", r);
    free(r);

    r=minWindow("a","b"); // ""
    printf("'%s'\n", r);
    free(r);

    return 0;
}


8) Longest Substring with K Distinct Characters
Description
Given a string s and an integer k, find the length of the longest substring of s that contains at most k distinct characters. If k=2, for example, and s="eceba", the answer is 3 (substring "ece").

Solution Approach
Sliding window + hash map to track how many distinct chars.
Expand end, track the count of each char.
If distinct chars > k, move start until distinct <= k.
Keep track of max window size.
Test Cases
s="eceba", k=2 => 3 ("ece")
s="aa", k=1 => 2
s="abc", k=3 => 3 (whole string)

In [8]:
def length_of_longest_k_distinct(s, k):
    from collections import defaultdict
    start=0
    max_len=0
    freq=defaultdict(int)
    distinct_count=0

    for end,ch in enumerate(s):
        freq[ch]+=1
        if freq[ch]==1:
            distinct_count+=1
        while distinct_count>k:
            left_char=s[start]
            freq[left_char]-=1
            if freq[left_char]==0:
                distinct_count-=1
            start+=1
        max_len=max(max_len, end-start+1)
    return max_len

if __name__ == "__main__":
    print(length_of_longest_k_distinct("eceba",2)) # 3
    print(length_of_longest_k_distinct("aa",1))    # 2
    print(length_of_longest_k_distinct("abc",3))   # 3


3
2
3


In [ ]:
#include <stdio.h>
#include <string.h>

int lengthOfLongestKDistinct(const char* s, int k){
    int freq[256]={0};
    int distinct=0;
    int start=0, maxLen=0;
    int n=strlen(s);
    for(int end=0;end<n;end++){
        unsigned char c=(unsigned char)s[end];
        freq[c]++;
        if(freq[c]==1) distinct++;
        while(distinct>k){
            unsigned char lc=(unsigned char)s[start];
            freq[lc]--;
            if(freq[lc]==0) distinct--;
            start++;
        }
        int length=end-start+1;
        if(length>maxLen) maxLen=length;
    }
    return maxLen;
}

int main(){
    printf("%d\n", lengthOfLongestKDistinct("eceba",2)); // 3
    printf("%d\n", lengthOfLongestKDistinct("aa",1));    // 2
    printf("%d\n", lengthOfLongestKDistinct("abc",3));   // 3
    return 0;
}


9) Find All Anagrams of a String in Another (a.k.a. “Find All Anagrams in a String”)
Description
Given two strings s and p, return all the start indices of substrings in s that are anagrams of p. If s="cbaebabacd", p="abc", result is [0,6] because "cba" and "bac" are anagrams of "abc".

Solution Approach
Similar to “Permutation in String”, but we collect all starting positions. Use a sliding window of length len(p) over s. Keep frequency counts, if match => record index.

Test Cases
s="cbaebabacd", p="abc" => [0, 6]
s="abab", p="ab" => [0,1,2]


In [9]:
def find_anagrams(s, p):
    from collections import Counter
    lenp = len(p)
    lens = len(s)
    if lenp>lens:
        return []
    res=[]
    countP=Counter(p)
    window=Counter(s[:lenp])
    if window==countP:
        res.append(0)
    for i in range(lenp, lens):
        window[s[i]] +=1
        window[s[i-lenp]] -=1
        if window[s[i-lenp]]==0:
            del window[s[i-lenp]]
        if window==countP:
            res.append(i-lenp+1)
    return res

if __name__ == "__main__":
    print(find_anagrams("cbaebabacd","abc")) # [0,6]
    print(find_anagrams("abab","ab"))        # [0,1,2]


[0, 6]
[0, 1, 2]


In [ ]:
#include <stdio.h>
#include <string.h>
#include <stdbool.h>

bool compareArray(int* a, int* b){
    for(int i=0;i<26;i++){
        if(a[i]!=b[i]) return false;
    }
    return true;
}

void findAnagrams(const char* s, const char* p){
    int ns=strlen(s), np=strlen(p);
    if(np>ns){
        printf("No anagrams\n");
        return;
    }
    int freqP[26]={0}, window[26]={0};
    for(int i=0;i<np;i++){
        freqP[p[i]-'a']++;
        window[s[i]-'a']++;
    }
    if(compareArray(freqP, window)){
        printf("0 ");
    }
    for(int i=np;i<ns;i++){
        window[s[i]-'a']++;
        window[s[i-np]-'a']--;
        if(compareArray(freqP, window)){
            printf("%d ", i-np+1);
        }
    }
    printf("\n");
}

int main(){
    findAnagrams("cbaebabacd","abc"); // 0 6
    findAnagrams("abab","ab");       // 0 1 2
    return 0;
}


10) Longest Palindromic Substring
Description
Find the longest palindromic substring in a given string s. E.g., s="babad" => longest palindrome is "bab" or "aba".

Solution Approach
Expand Around Center:

For each index i, expand around i (odd length) and i,i+1 (even length).
Keep track of max palindrome found.
Time Complexity: O(n^2)

Test Cases
s="babad" => "bab" or "aba"
s="cbbd" => "bb"
s="a" => "a"

In [10]:
def longest_palindrome(s):
    res=""
    for i in range(len(s)):
        # odd
        l=r=i
        while l>=0 and r<len(s) and s[l]==s[r]:
            if (r-l+1) > len(res):
                res = s[l:r+1]
            l-=1
            r+=1
        # even
        l=i
        r=i+1
        while l>=0 and r<len(s) and s[l]==s[r]:
            if (r-l+1)>len(res):
                res=s[l:r+1]
            l-=1
            r+=1
    return res

if __name__ == "__main__":
    print(longest_palindrome("babad")) # "bab" or "aba"
    print(longest_palindrome("cbbd"))  # "bb"
    print(longest_palindrome("a"))     # "a"


bab
bb
a


In [ ]:
#include <stdio.h>
#include <string.h>

void expand(const char* s,int left,int right,int* start,int* maxLen){
    int n=strlen(s);
    while(left>=0 && right<n && s[left]==s[right]){
        if(right-left+1>*maxLen){
            *maxLen=right-left+1;
            *start=left;
        }
        left--;
        right++;
    }
}

char* longestPalindrome(const char* s){
    int n=strlen(s);
    if(n<2) {
        char* r=(char*)malloc(n+1);
        strcpy(r,s);
        return r;
    }
    int start=0, maxLen=1;
    for(int i=0;i<n;i++){
        expand(s,i,i,&start,&maxLen);
        expand(s,i,i+1,&start,&maxLen);
    }
    char* res=(char*)malloc(maxLen+1);
    strncpy(res,s+start,maxLen);
    res[maxLen]='\0';
    return res;
}

int main(){
    char* r = longestPalindrome("babad"); // "bab" or "aba"
    printf("%s\n", r);
    free(r);

    r = longestPalindrome("cbbd"); // "bb"
    printf("%s\n", r);
    free(r);

    r = longestPalindrome("a"); // "a"
    printf("%s\n", r);
    free(r);

    return 0;
}


## Refresh

1) Permute a Linked List
Description
You have a singly linked list of unique integers. Generate all permutations (orderings) of the nodes in that list, returning each permutation as a newly formed linked list or as a list of values.

Example: For a linked list [1 -> 2 -> 3], produce:

[1,2,3], [1,3,2], [2,1,3], [2,3,1], [3,1,2], [3,2,1]

Solution Approach
Extract all the node values into an array.
Use standard backtracking to produce permutations.
For each permutation, either build a new linked list or just store the node values in an output.
Time Complexity: O(n * n!) to generate and build permutations.

Test Cases
Linked list: 1->2->3 => 6 permutations.
Linked list: 1->5 => 2 permutations ([1,5], [5,1]).
Linked list: (empty) => 1 permutation (empty).

In [11]:
class ListNode:
    def __init__(self, val=0, nxt=None):
        self.val = val
        self.next = nxt

def build_list(arr):
    head = None
    for val in reversed(arr):
        node = ListNode(val, head)
        head = node
    return head

def list_to_array(head):
    res = []
    while head:
        res.append(head.val)
        head = head.next
    return res

def permute_linkedlist(head):
    # 1) get all values
    arr = list_to_array(head)
    arr.sort()  # optional if you want permutations in sorted order
    results = []
    used = [False]*len(arr)

    def backtrack(path):
        if len(path) == len(arr):
            results.append(path[:])
            return
        for i in range(len(arr)):
            if used[i]:
                continue
            # skip duplicates if arr not unique
            # if i>0 and arr[i] == arr[i-1] and not used[i-1]: continue
            used[i] = True
            path.append(arr[i])
            backtrack(path)
            path.pop()
            used[i] = False

    backtrack([])
    return results

if __name__ == "__main__":
    # Test 1
    head1 = build_list([1,2,3])
    perms1 = permute_linkedlist(head1)
    print(perms1)
    # e.g. [[1,2,3],[1,3,2],[2,1,3],[2,3,1],[3,1,2],[3,2,1]]

    # Test 2
    head2 = build_list([1,5])
    perms2 = permute_linkedlist(head2)
    print(perms2)  # [[1,5],[5,1]]

    # Test 3 (empty)
    head3 = None
    perms3 = permute_linkedlist(head3)
    print(perms3)  # [[]]


[[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 3, 1], [3, 1, 2], [3, 2, 1]]
[[1, 5], [5, 1]]
[[]]


In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <stdbool.h>

// Linked list node
typedef struct ListNode {
    int val;
    struct ListNode* next;
} ListNode;

ListNode* buildList(int* arr, int n){
    ListNode* head=NULL;
    for(int i=n-1;i>=0;i--){
        ListNode* node=(ListNode*)malloc(sizeof(ListNode));
        node->val=arr[i];
        node->next=head;
        head=node;
    }
    return head;
}

int listToArray(ListNode* head, int* out){
    int cnt=0;
    while(head){
        out[cnt++]=head->val;
        head=head->next;
    }
    return cnt;
}

void swap(int* arr, int i,int j){
    int tmp=arr[i];
    arr[i]=arr[j];
    arr[j]=tmp;
}

void printPermutation(int* arr, int n){
    printf("[");
    for(int i=0;i<n;i++){
        printf("%d",arr[i]);
        if(i<n-1) printf(",");
    }
    printf("]");
}

void permuteUtil(int* arr,int start,int n){
    // if we want to simply print them:
    if(start==n){
        printPermutation(arr,n);
        printf(" ");
        return;
    }
    for(int i=start;i<n;i++){
        swap(arr, i, start);
        permuteUtil(arr, start+1, n);
        swap(arr, i, start);
    }
}

void permuteLinkedList(ListNode* head){
    // convert to array
    int buffer[100];
    int n=listToArray(head, buffer);
    // let's just do permutations using typical backtracking
    // to keep it simple, not skipping duplicates (assuming all unique)
    permuteUtil(buffer,0,n);
    printf("\n");
}

int main(){
    int arr1[]={1,2,3}; 
    ListNode* head1=buildList(arr1,3);
    permuteLinkedList(head1);
    
    int arr2[]={1,5};
    ListNode* head2=buildList(arr2,2);
    permuteLinkedList(head2);
    
    // empty
    ListNode* head3=NULL;
    permuteLinkedList(head3);
    
    return 0;
}


2) Shortest Path in a Maze (Backtracking)
Description
Given a 2D grid (maze) of cells that can be passable or blocked, find a shortest path from top-left (0,0) to bottom-right (m-1,n-1) if it exists, using only 4-directional moves. Use backtracking to explore paths. (Alternatively BFS is simpler, but we’ll do a backtracking approach for practice.)

Solution Approach (Backtracking version)
Maintain a global or external best_path.
Start DFS from (0,0), track visited cells.
Each step, move up/down/left/right if valid.
If you reach (m-1,n-1), compare path length with best; if smaller, update best.
Return best path if found, else an empty.
Time Complexity: Potentially O((m*n)!) in worst case for pure backtracking, but typically pruned.

Test Cases
Maze:
0 0 1
0 0 0
1 0 0
(0 => passable, 1=>blocked). A valid path might be length 4.
Single cell => trivial.
No path => return none.

In [12]:
def shortest_path_maze_backtrack(maze):
    if not maze or not maze[0]:
        return []
    m, n = len(maze), len(maze[0])
    best_path = [None]
    visited = [[False]*n for _ in range(m)]

    def dfs(r,c, path):
        if r==m-1 and c==n-1:
            # Reached goal
            if best_path[0] is None or len(path)<len(best_path[0]):
                best_path[0] = path[:]
            return
        # 4 directions
        for (dr,dc) in [(1,0),(-1,0),(0,1),(0,-1)]:
            nr, nc = r+dr, c+dc
            if 0<=nr<m and 0<=nc<n and maze[nr][nc]==0 and not visited[nr][nc]:
                visited[nr][nc]=True
                path.append((nr,nc))
                dfs(nr,nc,path)
                path.pop()
                visited[nr][nc]=False

    if maze[0][0]!=0:
        return []
    visited[0][0]=True
    dfs(0,0, [(0,0)])
    return best_path[0] if best_path[0] else []

if __name__=="__main__":
    maze1 = [
        [0,0,1],
        [0,0,0],
        [1,0,0]
    ]
    print(shortest_path_maze_backtrack(maze1))  # e.g. [(0,0),(0,1),(1,1),(1,2),(2,2)]


[(0, 0), (1, 0), (1, 1), (2, 1), (2, 2)]


In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <stdbool.h>

int bestLen=999999;
int** bestPath=NULL;
int bestCount=0;

void dfsMaze(int** maze, bool** visited, int m,int n, int r,int c, int** path,int length){
    // if reach end
    if(r==m-1 && c==n-1){
        // update best if length < bestLen
        if(length<bestLen){
            bestLen=length;
            // copy path
            if(bestPath){
                for(int i=0;i<bestCount;i++){
                    free(bestPath[i]);
                }
                free(bestPath);
            }
            bestCount=length;
            bestPath=(int**)malloc(sizeof(int*)*bestCount);
            for(int i=0;i<bestCount;i++){
                bestPath[i]=(int*)malloc(sizeof(int)*2);
                bestPath[i][0]=path[i][0];
                bestPath[i][1]=path[i][1];
            }
        }
        return;
    }
    // 4 directions
    int dirs[4][2]={{1,0},{-1,0},{0,1},{0,-1}};
    for(int i=0;i<4;i++){
        int nr=r+dirs[i][0], nc=c+dirs[i][1];
        if(nr>=0 && nr<m && nc>=0 && nc<n && maze[nr][nc]==0 && !visited[nr][nc]){
            visited[nr][nc]=true;
            path[length][0]=nr; 
            path[length][1]=nc;
            dfsMaze(maze,visited,m,n,nr,nc,path,length+1);
            visited[nr][nc]=false;
        }
    }
}

int main(){
    int mazeArr[3][3]={
        {0,0,1},
        {0,0,0},
        {1,0,0}
    };
    int m=3,n=3;
    int** maze=(int**)malloc(m*sizeof(int*));
    for(int i=0;i<m;i++){
        maze[i]=(int*)malloc(n*sizeof(int));
        for(int j=0;j<n;j++){
            maze[i][j]=mazeArr[i][j];
        }
    }
    bool** visited=(bool**)malloc(m*sizeof(bool*));
    for(int i=0;i<m;i++){
        visited[i]=(bool*)calloc(n,sizeof(bool));
    }
    // path array
    int** path=(int**)malloc((m*n)*sizeof(int*));
    for(int i=0;i<m*n;i++){
        path[i]=(int*)malloc(2*sizeof(int));
    }
    if(maze[0][0]==0){
        visited[0][0]=true;
        path[0][0]=0; path[0][1]=0;
        dfsMaze(maze,visited,m,n,0,0,path,1);
    }
    if(bestLen==999999){
        printf("No path found\n");
    } else {
        printf("Shortest path length = %d\n", bestLen);
        for(int i=0;i<bestCount;i++){
            printf("(%d,%d) ", bestPath[i][0], bestPath[i][1]);
        }
        printf("\n");
    }
    // free ...
    return 0;
}


3) All Combinations of Balanced Parentheses
Description
Given an integer n, generate all strings of length 2n consisting of '(' and ')' that are valid balanced parentheses sequences.

Example (n=3):

arduino

[
  "((()))",
  "(()())",
  "(())()",
  "()(())",
  "()()()"
]
Solution Approach
Use backtracking with two counters:

open = how many '(' used so far
close = how many ')' used so far
At each step, we can add '(' if open < n; add ')' if close < open.
Time Complexity: O(C(2n,n)) ~ Catalan number.

Test Cases
n=1 => ["()"]
n=2 => ["(())","()()"]
n=3 => as above

In [13]:
def generate_parenthesis(n):
    res=[]
    def backtrack(path, openCount, closeCount):
        if len(path)==2*n:
            res.append(path)
            return
        if openCount<n:
            backtrack(path+"(", openCount+1, closeCount)
        if closeCount<openCount:
            backtrack(path+")", openCount, closeCount+1)
    backtrack("",0,0)
    return res

if __name__=="__main__":
    print(generate_parenthesis(3))
    print(generate_parenthesis(1))


['((()))', '(()())', '(())()', '()(())', '()()()']
['()']


In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

char** ans;
int ansSize=0;

void backtrackParenthesis(int n, int open, int close, char* path, int index){
    if(index==2*n){
        path[index]='\0';
        ans[ansSize] = (char*)malloc((2*n+1)*sizeof(char));
        strcpy(ans[ansSize], path);
        ansSize++;
        return;
    }
    if(open<n){
        path[index]='(';
        backtrackParenthesis(n, open+1, close, path, index+1);
    }
    if(close<open){
        path[index]=')';
        backtrackParenthesis(n, open, close+1, path, index+1);
    }
}

int main(){
    int n=3;
    // upper bound for # of results = Catalan number. For n=3, it's 5.
    ans = (char**)malloc(100*sizeof(char*));
    ansSize=0;
    char path[100];
    backtrackParenthesis(n,0,0,path,0);
    for(int i=0;i<ansSize;i++){
        printf("%s\n", ans[i]);
        free(ans[i]);
    }
    free(ans);
    return 0;
}

4) All Subsets (Power Set) Handling Duplicates
Description
Given an array (possibly containing duplicates), find all unique subsets (the power set). Return them in any order, but typically sorted.

Example: [1,2,2] => subsets:

css
복사
편집
[], [1], [2], [1,2], [2,2], [1,2,2]
Solution Approach
Sort array.
Backtracking to build subsets: for each index i, decide include or not include.
Skip duplicates: if nums[i]==nums[i-1] and not used i-1, skip.
Time Complexity: O(2^n) * O(n) for storing subsets, plus duplicates skip.

Test Cases
[1,2,2] => 6 subsets
[0] => [[],[0]]
[1,1,1] => [[],[1],[1,1],[1,1,1]]

In [14]:
def subsets_with_dup(nums):
    nums.sort()
    res=[]
    subset=[]
    def backtrack(start):
        res.append(subset[:])
        for i in range(start,len(nums)):
            if i>start and nums[i]==nums[i-1]:
                continue
            subset.append(nums[i])
            backtrack(i+1)
            subset.pop()
    backtrack(0)
    return res

if __name__=="__main__":
    print(subsets_with_dup([1,2,2]))
    # [[],[1],[1,2],[1,2,2],[2],[2,2]]


[[], [1], [1, 2], [1, 2, 2], [2], [2, 2]]


In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <stdbool.h>

int cmp(const void* a,const void* b){return (*(int*)a - *(int*)b);}
int path[100];
int** ans;
int ansSize=0;
int* colSize;

void backtrackSubsetDup(int* nums,int n,int start,int depth){
    // store current subset
    ans[ansSize]=(int*)malloc(depth*sizeof(int));
    for(int i=0;i<depth;i++){
        ans[ansSize][i]=path[i];
    }
    colSize[ansSize]=depth;
    ansSize++;

    for(int i=start;i<n;i++){
        if(i>start && nums[i]==nums[i-1]) continue;
        path[depth]=nums[i];
        backtrackSubsetDup(nums,n,i+1,depth+1);
    }
}

int main(){
    int arr[]={1,2,2};
    int n=3;
    qsort(arr,n,sizeof(int),cmp);

    ans=(int**)malloc(1000*sizeof(int*));
    colSize=(int*)malloc(1000*sizeof(int));
    ansSize=0;
    backtrackSubsetDup(arr,n,0,0);
    for(int i=0;i<ansSize;i++){
        printf("[");
        for(int j=0;j<colSize[i];j++){
            printf("%d", ans[i][j]);
            if(j<colSize[i]-1) printf(",");
        }
        printf("]\n");
        free(ans[i]);
    }
    free(ans);
    free(colSize);
    return 0;
}

5) Generate Palindromic Decompositions of a String
Description
Given a string s, partition it into substrings such that each substring is a palindrome. Return all possible such decompositions.

Example: s="aab" =>

["a","a","b"]
["aa","b"]
Solution Approach
Use backtracking:

Recursively try all splits.
Check if substring s[start:i+1] is palindrome. If yes, recurse from i+1.
Test Cases
"aab" => [["a","a","b"], ["aa","b"]]
"aaa" => [["a","a","a"],["aa","a"],["a","aa"],["aaa"]]

In [15]:
def partition_palindrome(s):
    res=[]
    def is_palindrome(st):
        return st==st[::-1]
    def backtrack(start, path):
        if start==len(s):
            res.append(path[:])
            return
        for i in range(start,len(s)):
            substr=s[start:i+1]
            if is_palindrome(substr):
                path.append(substr)
                backtrack(i+1,path)
                path.pop()
    backtrack(0,[])
    return res

if __name__=="__main__":
    print(partition_palindrome("aab"))
    # [["a","a","b"],["aa","b"]]


[['a', 'a', 'b'], ['aa', 'b']]


In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <stdbool.h>

bool isPalindrome(const char* s, int start,int end){
    while(start<end){
        if(s[start]!=s[end]) return false;
        start++; end--;
    }
    return true;
}

char*** ans; 
int ansSize=0;
int* colSize;

char path[100][100]; 
int pathLen=0;

void backtrackPartition(const char* s, int start, int n){
    if(start==n){
        // store path
        ans[ansSize]=(char**)malloc(pathLen*sizeof(char*));
        for(int i=0;i<pathLen;i++){
            ans[ansSize][i]=(char*)malloc(strlen(path[i])+1);
            strcpy(ans[ansSize][i], path[i]);
        }
        colSize[ansSize]=pathLen;
        ansSize++;
        return;
    }
    for(int i=start;i<n;i++){
        if(isPalindrome(s,start,i)){
            // store substring s[start..i]
            int length=i-start+1;
            strncpy(path[pathLen], s+start,length);
            path[pathLen][length]='\0';
            pathLen++;
            backtrackPartition(s,i+1,n);
            pathLen--;
        }
    }
}

int main(){
    char s[]="aab";
    ans=(char***)malloc(1000*sizeof(char**));
    colSize=(int*)malloc(1000*sizeof(int));
    ansSize=0; pathLen=0;
    backtrackPartition(s,0,strlen(s));
    
    for(int i=0;i<ansSize;i++){
        printf("[");
        for(int j=0;j<colSize[i];j++){
            printf("%s", ans[i][j]);
            if(j<colSize[i]-1) printf(",");
        }
        printf("]\n");
        // free allocated
    }
    return 0;
}


6) Restore IP Addresses
Description
Given a string of digits, return all possible valid IPv4 addresses that can be formed by inserting . in the string. Each segment must be 0–255, no leading zeros unless “0” alone.

Example: "25525511135" =>

"255.255.11.135", "255.255.111.35", etc.
Solution Approach
Use backtracking with up to 4 segments. Each segment can be length 1–3, must parse to 0–255, no leading zero unless the segment is “0”.

Test Cases
"25525511135" => ["255.255.11.135","255.255.111.35"]
"0000" => ["0.0.0.0"]
"1111" => ["1.1.1.1"]

In [16]:
def restore_ip_addresses(s):
    res=[]
    def backtrack(index, path, segs):
        if segs==4:
            if index==len(s):
                res.append(".".join(path))
            return
        # pick 1-3 digits
        for length in range(1,4):
            if index+length>len(s):
                break
            segment = s[index:index+length]
            # check valid
            if (segment[0]=='0' and len(segment)>1) or int(segment)>255:
                continue
            path.append(segment)
            backtrack(index+length,path,segs+1)
            path.pop()
    backtrack(0,[],0)
    return res

if __name__=="__main__":
    print(restore_ip_addresses("25525511135"))
    # ["255.255.11.135","255.255.111.35"]


['255.255.11.135', '255.255.111.35']


In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <stdbool.h>

char** ans;
int ansSize=0;

bool validSegment(const char* seg){
    int len=strlen(seg);
    if(len==0||len>3) return false;
    if(seg[0]=='0' && len>1) return false;
    int val=atoi(seg);
    return (val>=0 && val<=255);
}

void backtrackIP(const char* s,int start,int segs,char* path,int pathLen){
    int n=strlen(s);
    if(segs==4){
        if(start==n){
            path[pathLen]='\0';
            ans[ansSize]=(char*)malloc(pathLen+1);
            strcpy(ans[ansSize], path);
            ansSize++;
        }
        return;
    }
    for(int length=1; length<=3; length++){
        if(start+length>n) break;
        char segment[4];
        strncpy(segment, s+start, length);
        segment[length]='\0';
        if(!validSegment(segment)) continue;
        int addLen=length;
        // place segment
        for(int i=0;i<length;i++){
            path[pathLen+i]=segment[i];
        }
        if(segs<3){
            path[pathLen+addLen]='.';
            backtrackIP(s,start+length,segs+1,path,pathLen+addLen+1);
        } else {
            // no dot if last segment
            backtrackIP(s,start+length,segs+1,path,pathLen+addLen);
        }
    }
}

int main(){
    char s[]="25525511135";
    ans=(char**)malloc(1000*sizeof(char*));
    ansSize=0;
    char path[30];
    backtrackIP(s,0,0,path,0);
    for(int i=0;i<ansSize;i++){
        printf("%s\n", ans[i]);
        free(ans[i]);
    }
    free(ans);
    return 0;
}


7) Word Break II (All possible segmentations)
Description
Given a string s and a dictionary of words, return all possible ways to segment s such that each segment is a valid word in the dictionary.

Example: s="catsanddog", dict=["cat","cats","and","sand","dog"] Output:

arduino
복사
편집
[
  "cats and dog",
  "cat sand dog"
]
Solution Approach
Backtracking over the string, at each index try to pick a dictionary word that matches a prefix, then recurse. Usually we add memoization to optimize.

Test Cases
s="catsanddog", dict=["cat","cats","and","sand","dog"] => 2 ways.
s="pineapplepenapple", dict=["apple","pen","applepen","pine","pineapple"] => 3 ways.
s="abcd", dict does not allow => none.

In [17]:
def word_break_all(s, wordDict):
    wordSet = set(wordDict)
    memo = {}
    def backtrack(start):
        if start in memo:
            return memo[start]
        if start==len(s):
            return [""]  # means valid ended
        res=[]
        for end in range(start+1,len(s)+1):
            prefix=s[start:end]
            if prefix in wordSet:
                sublist = backtrack(end)
                for sub in sublist:
                    if sub=="":
                        res.append(prefix)
                    else:
                        res.append(prefix+" "+sub)
        memo[start]=res
        return res
    return backtrack(0)

if __name__=="__main__":
    print(word_break_all("catsanddog",["cat","cats","and","sand","dog"]))
    # ["cats and dog","cat sand dog"]


['cat sand dog', 'cats and dog']


In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <stdbool.h>

typedef struct {
    char** arr;
    int size;
    int cap;
} StrList;

bool inDict(char** dict,int dictSize,const char* word){
    for(int i=0;i<dictSize;i++){
        if(strcmp(dict[i],word)==0) return true;
    }
    return false;
}

void addStr(StrList* list,const char* s){
    if(list->size==list->cap){
        list->cap*=2;
        list->arr=(char**)realloc(list->arr,list->cap*sizeof(char*));
    }
    list->arr[list->size]=(char*)malloc(strlen(s)+1);
    strcpy(list->arr[list->size],s);
    list->size++;
}

void wordBreakDfs(const char* s,int idx,char** dict,int dictSize,StrList* res,char* path,int pathLen){
    int n=strlen(s);
    if(idx==n){
        path[pathLen]='\0';
        addStr(res,path);
        return;
    }
    for(int end=idx+1;end<=n;end++){
        char temp[256];
        int length=end-idx;
        strncpy(temp, s+idx,length);
        temp[length]='\0';
        if(inDict(dict,dictSize,temp)){
            // add temp + " "
            int oldPathLen=pathLen;
            if(pathLen>0) {
                path[pathLen]=' ';
                pathLen++;
            }
            strcpy(&path[pathLen], temp);
            int newPathLen=pathLen+strlen(temp);
            wordBreakDfs(s,end,dict,dictSize,res,path,newPathLen);
            pathLen=oldPathLen; 
            path[pathLen]='\0';
        }
    }
}

int main(){
    char s[]="catsanddog";
    char* dict[]={"cat","cats","and","sand","dog"};
    int dictSize=5;
    StrList res;
    res.size=0; res.cap=10;
    res.arr=(char**)malloc(res.cap*sizeof(char*));
    char path[256]="";
    wordBreakDfs(s,0,dict,dictSize,&res,path,0);
    for(int i=0;i<res.size;i++){
        printf("%s\n", res.arr[i]);
        free(res.arr[i]);
    }
    free(res.arr);
    return 0;
}


8) N-Queens Problem
Description
Place n queens on an n x n chessboard such that no two queens can attack each other. Return all distinct solutions, each solution is a board arrangement.

Example: n=4 => 2 solutions:

arduino

[
  [".Q..","...Q","Q...","..Q."],
  ["..Q.","Q...","...Q",".Q.."]
]
Solution Approach
Backtracking row by row. Keep track of columns, diagonals used.

Test Cases
n=4 => 2 solutions
n=1 => 1 solution [["Q"]]
n=2 => 0 solutions

In [18]:
def solveNQueens(n):
    res=[]
    board=[["."]*n for _ in range(n)]
    cols=set()
    d1=set() # r-c
    d2=set() # r+c

    def backtrack(r):
        if r==n:
            solution=["".join(row) for row in board]
            res.append(solution)
            return
        for c in range(n):
            if c in cols or (r-c) in d1 or (r+c) in d2:
                continue
            board[r][c]="Q"
            cols.add(c)
            d1.add(r-c)
            d2.add(r+c)
            backtrack(r+1)
            board[r][c]="."
            cols.remove(c)
            d1.remove(r-c)
            d2.remove(r+c)
    backtrack(0)
    return res

if __name__=="__main__":
    print(solveNQueens(4))


[['.Q..', '...Q', 'Q...', '..Q.'], ['..Q.', 'Q...', '...Q', '.Q..']]


In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <stdbool.h>

char*** ans; 
int ansSize=0;
int* colSizes;
bool colUsed[50];
bool diag1[100]; // r-c up to +/- n
bool diag2[100]; // r+c up to 2n

void backtrackNQ(int n,int r,char** board){
    if(r==n){
        // store solution
        ans[ansSize]=(char**)malloc(n*sizeof(char*));
        for(int i=0;i<n;i++){
            ans[ansSize][i]=(char*)malloc((n+1)*sizeof(char));
            strcpy(ans[ansSize][i],board[i]);
        }
        colSizes[ansSize]=n;
        ansSize++;
        return;
    }
    for(int c=0;c<n;c++){
        if(colUsed[c]) continue;
        int d1=r-c+n; 
        int d2=r+c;
        if(diag1[d1]||diag2[d2]) continue;
        board[r][c]='Q';
        colUsed[c]=true; diag1[d1]=true; diag2[d2]=true;
        backtrackNQ(n,r+1,board);
        board[r][c]='.';
        colUsed[c]=false; diag1[d1]=false; diag2[d2]=false;
    }
}

int main(){
    int n=4;
    ans=(char***)malloc(1000*sizeof(char**));
    colSizes=(int*)malloc(1000*sizeof(int));
    ansSize=0;
    memset(colUsed,false,sizeof(colUsed));
    memset(diag1,false,sizeof(diag1));
    memset(diag2,false,sizeof(diag2));

    char** board=(char**)malloc(n*sizeof(char*));
    for(int i=0;i<n;i++){
        board[i]=(char*)malloc((n+1)*sizeof(char));
        for(int j=0;j<n;j++){
            board[i][j]='.';
        }
        board[i][n]='\0';
    }
    backtrackNQ(n,0,board);

    printf("Number of solutions: %d\n",ansSize);
    for(int i=0;i<ansSize;i++){
        printf("Solution %d:\n", i+1);
        for(int r=0;r<colSizes[i];r++){
            printf("%s\n", ans[i][r]);
        }
        printf("\n");
    }
    return 0;
}


9) Partition to K Equal Sum Subsets
Description
Given an integer array nums and an integer k, check if it is possible to partition nums into k subsets with equal sum. Return any one partitioning or just a boolean indicating if possible.

Example:

nums=[4,3,2,3,5,2,1], k=4 => true, because [4],[3,1],[2,2],[3,5] each sum to 5.
Solution Approach
sum(nums) must be divisible by k => target = total/k. --> 나눠지는 거 찾나;
Use backtracking to fill subsets. Sort nums descending for better pruning.
Recurse or maintain a used[] and a subsetSum[k].
Test Cases
[4,3,2,3,5,2,1], k=4 => true
[1,2,3], k=2 => false if sum not divisible by 2.
Large arrays with no solution.

In [19]:
def canPartitionKSubsets(nums, k):
    s=sum(nums)
    if s%k!=0: return False
    target=s//k
    nums.sort(reverse=True)
    used=[False]*len(nums)
    buckets=[0]*k

    def backtrack(index):
        if index==len(nums):
            return True
        for b in range(k):
            if buckets[b]+nums[index]<=target:
                buckets[b]+=nums[index]
                if backtrack(index+1):
                    return True
                buckets[b]-=nums[index]
            if buckets[b]==0: 
                break
        return False

    return backtrack(0)

if __name__=="__main__":
    print(canPartitionKSubsets([4,3,2,3,5,2,1],4))  # true


True


In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <stdbool.h>

int cmpDesc(const void* a, const void* b){return (*(int*)b - *(int*)a;}

bool backtrackK(int* nums,int n,int* buckets,int k,int target,int idx){
    if(idx==n) return true;
    for(int i=0;i<k;i++){
        if(buckets[i]+nums[idx]<=target){
            buckets[i]+=nums[idx];
            if(backtrackK(nums,n,buckets,k,target,idx+1)) return true;
            buckets[i]-=nums[idx];
        }
        if(buckets[i]==0) break;
    }
    return false;
}

bool canPartitionKSubsets(int* nums,int n,int k){
    int sum=0;
    for(int i=0;i<n;i++) sum+=nums[i];
    if(sum%k!=0) return false;
    int target=sum/k;
    qsort(nums,n,sizeof(int),cmpDesc);
    int* buckets=(int*)calloc(k,sizeof(int));
    bool ans=backtrackK(nums,n,buckets,k,target,0);
    free(buckets);
    return ans;
}

int main(){
    int arr[]={4,3,2,3,5,2,1};
    int n=7, k=4;
    printf("%d\n", canPartitionKSubsets(arr,n,k)); // 1 (true)
    return 0;
}


10) Expression Add Operators
Description
Given a string num of digits and a target integer, add binary operators +, -, or * between the digits so that the expression evaluates to target. Return all valid expressions.

Example:

num="123", target=6 => ["1+2+3","1*2+3"]
num="232", target=8 => ["2*3+2","2+3*2"]
Solution Approach
Use backtracking over string num, at each position choose to put '+'/'-'/'*' or no operator if it’s the first number.
Keep track of the current evaluation and also the last factor to handle multiplication.
If we reach end of string and eval == target, add expression to results.
Time Complexity: O(4^n) in worst case.

Test Cases
"123", target=6 => ["1+2+3","1*2+3"]
"232", target=8 => ["2*3+2","2+3*2"]
"105", target=5 => ["1*0+5","10-5"]


In [ ]:
def addOperators(num, target):
    res=[]
    def backtrack(idx, expr, currVal, lastFactor):
        if idx==len(num):
            if currVal==target:
                res.append(expr)
            return
        for i in range(idx,len(num)):
            if i>idx and num[idx]=='0':
                break
            part=num[idx:i+1]
            val=int(part)
            if idx==0:
                backtrack(i+1, part, val, val)
            else:
                # plus
                backtrack(i+1, expr+"+"+part, currVal+val, val)
                # minus
                backtrack(i+1, expr+"-"+part, currVal-val, -val)
                # multiply
                backtrack(i+1, expr+"*"+part, currVal - lastFactor + lastFactor*val, lastFactor*val)
    backtrack(0,"",0,0)
    return res

if __name__=="__main__":
    print(addOperators("123",6))
    print(addOperators("232",8))


In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <stdbool.h>

char** ans;
int ansSize=0;

void addToAns(char* expr){
    ans[ansSize]=(char*)malloc(strlen(expr)+1);
    strcpy(ans[ansSize], expr);
    ansSize++;
}

void backtrack(const char* num,int target,int idx,long currVal,long lastFactor,char* expr,int exprLen){
    int n=strlen(num);
    if(idx==n){
        if(currVal==target){
            expr[exprLen]='\0';
            addToAns(expr);
        }
        return;
    }
    for(int i=idx;i<n;i++){
        // leading zero check
        if(i>idx && num[idx]=='0') break;
        char part[20];
        int length=i-idx+1;
        strncpy(part,num+idx,length);
        part[length]='\0';
        long val=atol(part);
        int oldLen=exprLen;
        if(idx==0){
            // first number
            sprintf(expr+exprLen,"%s",part);
            backtrack(num,target,i+1,val,val,expr,exprLen+length);
            exprLen=oldLen;
            expr[exprLen]='\0';
        } else {
            // +
            sprintf(expr+exprLen,"+%s",part);
            backtrack(num,target,i+1,currVal+val,val,expr,exprLen+length+1);
            exprLen=oldLen;
            expr[exprLen]='\0';
            // -
            sprintf(expr+exprLen,"-%s",part);
            backtrack(num,target,i+1,currVal-val,-val,expr,exprLen+length+1);
            exprLen=oldLen;
            expr[exprLen]='\0';
            // *
            long newVal=currVal - lastFactor + lastFactor*val;
            sprintf(expr+exprLen,"*%s",part);
            backtrack(num,target,i+1,newVal,lastFactor*val,expr,exprLen+length+1);
            exprLen=oldLen;
            expr[exprLen]='\0';
        }
    }
}

int main(){
    ans=(char**)malloc(1000*sizeof(char*));
    ansSize=0;
    char s[]="123";
    char buffer[1000];
    backtrack(s,6,0,0,0,buffer,0);
    for(int i=0;i<ansSize;i++){
        printf("%s\n", ans[i]);
        free(ans[i]);
    }
    free(ans);
    return 0;
}
